In [15]:
#Import the libraries needed
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests
import json

In [2]:
#Load dataframe of interest
city_data_df=pd.read_csv("../WeatherPy/analysis_data.csv")
city_data_df

,City,Country Code,Lat [deg],Long [deg],Temperature [F],Humidity [%],Cloudiness [%],Wind Speed [mph]
0,rikitea,pf,-23.12,-134.97,77.036,76,19,4.69
1,busselton,au,-33.65,115.33,81.428,29,100,7.54
2,punta arenas,cl,-53.15,-70.92,50.000,66,78,8.70
3,ushuaia,ar,-54.80,-68.30,53.600,91,40,6.20
4,hobart,au,-42.88,147.33,59.486,47,75,6.20
...,...,...,...,...,...,...,...,...
710,dikson,ru,73.51,80.55,30.056,96,100,7.71
711,college,us,64.86,-147.80,12.758,71,1,1.04
712,havoysund,no,71.00,24.66,15.800,85,100,11.80
713,narsaq,gl,60.92,-46.05,19.400,79,98,0.59


In [3]:
 # Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
 # Store 'Lat [deg]' and 'Long [deg]' into  locations 
locations_df = city_data_df[["Lat [deg]", "Long [deg]"]].astype(float)

# Convert Humidity to float and store
humidity_df = city_data_df["Humidity [%]"].astype(float)

In [73]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False, 
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Filter ideal weather conditions
cities_of_interest=pd.DataFrame()

for index,row in city_data_df.iterrows():
    if row["Temperature [F]"]>73 and row["Temperature [F]"]<78 and row["Wind Speed [mph]"]<5 and row["Cloudiness [%]"]==0:
        cities_of_interest[index]=row
#     else:
#         cities_of_interest
cities_of_interest.dropna().T


,City,Country Code,Lat [deg],Long [deg],Temperature [F],Humidity [%],Cloudiness [%],Wind Speed [mph]
27,arraial do cabo,br,-22.97,-42.03,77.09,85,0,4.46
89,ellisras,za,-23.66,27.74,76.082,56,0,3.3
223,san patricio,mx,19.22,-104.7,75.02,85,0,2.43
304,niamey,ne,13.51,2.11,75.2,14,0,2.1
329,santa maria,cv,-29.68,-53.81,74.21,68,0,1.91
343,dwarka,in,22.24,68.97,77.234,49,0,4.58
376,dali,cn,25.7,100.18,73.31,21,0,2.82


In [53]:
#Generate dataframe for nearest hotels
nearest_hotels_df=cities_of_interest.loc[["City","Country Code","Lat [deg]","Long [deg]"]].T
nearest_hotels_df["Hotel Name"]=""
nearest_hotels_df["Hotel Lat"]=""
nearest_hotels_df["Hotel Lng"]=""
nearest_hotels_df["Hotel Rating"]=""
nearest_hotels_df["Hotel Total Reviews"]=""
nearest_hotels_df.head()

,City,Country Code,Lat [deg],Long [deg],Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating,Hotel Total Reviews
27,arraial do cabo,br,-22.97,-42.03,,,,,
89,ellisras,za,-23.66,27.74,,,,,
223,san patricio,mx,19.22,-104.7,,,,,
304,niamey,ne,13.51,2.11,,,,,
329,santa maria,cv,-29.68,-53.81,,,,,


In [54]:
#Populate dataframe with nearest hotels
nearby_search_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in nearest_hotels_df.iterrows():
    
    #defining search parameters and performing request
    params = {
    "location": " {} , {} ".format(row["Lat [deg]"],row["Long [deg]"]) ,
    "rankby": "distance",
    "type": "lodging",
    "keyword": "hotel",
    "key": gkey}
    response = requests.get(nearby_search_url, params=params).json()
    
    #populating dataframe
    nearest_hotels_df.loc[index,"Hotel Lat"]=response["results"][0]['geometry']['location']["lat"]
    nearest_hotels_df.loc[index,"Hotel Lng"]=response["results"][0]['geometry']['location']['lng']
    nearest_hotels_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    nearest_hotels_df.loc[index,"Hotel Rating"]=response["results"][0]["rating"]
    nearest_hotels_df.loc[index,"Hotel Total Reviews"]=response["results"][0]["user_ratings_total"]

nearest_hotels_df

,City,Country Code,Lat [deg],Long [deg],Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating,Hotel Total Reviews
27,arraial do cabo,br,-22.97,-42.03,Pousada Caravela,-22.9703,-42.0292,4.2,684
89,ellisras,za,-23.66,27.74,Thaba'Nkwe Bush Lodge and Restaurant,-23.6665,27.7448,3.8,27
223,san patricio,mx,19.22,-104.7,Bungalows Mayorca,19.2211,-104.699,4.3,140
304,niamey,ne,13.51,2.11,Djoliba Lodge,13.5074,2.11305,4.2,13
329,santa maria,cv,-29.68,-53.81,Hotel Dom Rafael - Premium,-29.6794,-53.81,4.5,585
343,dwarka,in,22.24,68.97,Hotel Shree Nath,22.24,68.9687,3.8,78
376,dali,cn,25.7,100.18,Regent Hotel - Dali,25.6989,100.173,4.3,4


In [74]:
#Add hotel locations to heat map
hotel_loc_tuples=[]
for index,row in nearest_hotels_df.iterrows():
    hotel_loc_tuples.append( tuple([row["Hotel Lat"],row["Hotel Lng"]] )) 

markers=gmaps.marker_layer(hotel_loc_tuples)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))